<h1 style = "color:red; border-bottom: 4px solid gold; 
           align: center;
           padding-bottom: 5px;">Naive Bayes Tutorial</h1>

We call it naive, becaue we make the assumption of conditional independence of the xi's given Y


### Naïve Bayes Classifier — Conditional Independence

In the **Naïve Bayes classifier**, the key assumption is that the features $ X_1, X_2, \ldots, X_n $ are **conditionally independent** given the class $ Y $.  

Mathematically, this is written as:

$$
P(X_1, X_2, \ldots, X_n \mid Y) = \prod_{i=1}^{n} P(X_i \mid Y)
$$


This means that each feature $ X_i $ contributes **independently** to the probability of \( Y \), given \( Y \).  

---

### Conditional Probability Expression

The general conditional probability formula for Naïve Bayes can be written as:

$$
P(Y \mid X_1, X_2, \ldots, X_n) = \frac{P(X_1 \mid Y) \, P(X_2 \mid Y) \, \ldots \, P(X_n \mid Y)}{P(X_1) \, P(X_2) \, \ldots \, P(X_n)}
$$

---

### Posterior Probability Used for Prediction

In practice, we often compute the posterior probability as:

$$
P(Y \mid X_1, X_2, \ldots, X_n) = \frac{P(Y) \prod_{i=1}^{n} P(X_i \mid Y)}{P(X_1, X_2, \ldots, X_n)}
$$

Since the denominator $ P(X_1, X_2, \ldots, X_n) $ is constant across all classes, it can be ignored during classification.  
Thus, we use the proportional form:

$$
P(Y \mid X_1, X_2, \ldots, X_n) \propto P(Y) \prod_{i=1}^{n} P(X_i \mid Y)
$$


---
CategoricalNB → Use when features are categorical (nominal), like gender, color, region.

GaussianNB → Use when features are continuous and assumed to follow a Gaussian (normal) distribution, like height, weight, temperature.

MultinomialNB → Use when features represent counts or frequencies, like word counts in documents.

BernoulliNB → Use when features are binary (presence/absence), like word presence/absence in text.

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer



In [2]:
data = pd.DataFrame({
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Mild', 'Cool'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'High', 'Normal'],
    'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak'],
    'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes']
})

data['Temperature'] = np.random.uniform(60, 90, size=len(data))

# Introduce one missing value
data.loc[3, 'Temperature'] = np.nan

X = data[['Outlook', 'Temperature', 'Humidity', 'Wind']]
y = data['Play']

In [3]:
data.head()

,Outlook,Temperature,Humidity,Wind,Play
0,Sunny,85.098952,High,Weak,No
1,Sunny,73.512887,High,Strong,No
2,Overcast,60.691997,High,Weak,Yes
3,Rain,NaN,High,Weak,Yes
4,Rain,65.788286,Normal,Weak,Yes


In [4]:
data.isnull().sum()

Outlook        0
Temperature    1
Humidity       0
Wind           0
Play           0
dtype: int64

In [5]:
data.Play = data.Play.apply(lambda x: 1 if x=="Yes"  else 0)

In [6]:
transformer = ColumnTransformer(transformers=[
("T1",SimpleImputer(strategy="mean"),["Temperature"]),  ## Impute missing values in Temperature with mean
("T2", OrdinalEncoder(),["Wind","Humidity"]),   # Encode ordinal/categorical features (Wind, Humidity) numerically
("T3",OneHotEncoder( drop="first"), ["Outlook"])
],remainder="passthrough"             
)

transformer.fit_transform(data)

array([[85.09895219,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [73.51288742,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ],
       [60.69199729,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [76.94265148,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ],
       [65.78828632,  1.        ,  1.        ,  1.        ,  0.        ,
         1.        ],
       [86.45134712,  0.        ,  1.        ,  1.        ,  0.        ,
         0.        ],
       [79.89003091,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [87.16505911,  1.        ,  1.        ,  0.        ,  1.        ,
         1.        ]])

In [7]:
# Combine preprocessing + model
model = make_pipeline(transformer,GaussianNB())

# Separate features and target
X = data.drop(columns="Play")
y = data['Play']

# Fit the model
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('T1', SimpleImputer(),
                                                  ['Temperature']),
                                                 ('T2', OrdinalEncoder(),
                                                  ['Wind', 'Humidity']),
                                                 ('T3',
                                                  OneHotEncoder(drop='first'),
                                                  ['Outlook'])])),
                ('gaussiannb', GaussianNB())])

In [8]:
# Optional: make a prediction
prediction = model.predict(X)
print("Predictions:", prediction)

Predictions: [0 0 1 1 1 0 0 0]


In [9]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [10]:
#Drop unneccesary features in the dataset

df.drop(["PassengerId",	"Name", "SibSp"	,"Parch"	,"Ticket", "Cabin","Embarked"	], axis =1, inplace=True)

In [11]:
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [12]:
target = df.Survived
inputs = df.drop(["Survived"], axis=1)

In [13]:
dummies = pd.get_dummies(df.Sex).astype("int")
dummies

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
886,0,1
887,1,0
888,1,0
889,0,1


In [14]:
inputs = pd.concat([inputs,dummies],axis=1)
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


In [15]:
### Check for NA's in the data

inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [16]:
inputs.Age[1:10]

1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [17]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(inputs.drop(["Sex","female"],axis=1),target,test_size=0.2)


In [19]:
len(X_train)

712

In [20]:
from sklearn.naive_bayes import GaussianNB

In [21]:
model = GaussianNB()
model.fit(X_train,y_train)

GaussianNB()

In [22]:
model.score(X_test,y_test)

0.776536312849162

In [24]:
model.predict_proba(X_test)[1:10]

array([[0.96484597, 0.03515403],
       [0.22504723, 0.77495277],
       [0.96252557, 0.03747443],
       [0.02031512, 0.97968488],
       [0.36894301, 0.63105699],
       [0.22492337, 0.77507663],
       [0.95816169, 0.04183831],
       [0.02551446, 0.97448554],
       [0.21153584, 0.78846416]])